<a href="https://colab.research.google.com/github/bjpark-forest/2023-1-Intro_ML/blob/main/Ensemble_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensembling

In [ ]:
from sklearn import datasets

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# three base models
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.ensemble import VotingClassifier


In [ ]:
import requests
import io
import pandas as pd


# 임웅교수님 공유 링크에서 파일 ID 추출
file_url = "https://drive.google.com/file/d/1xKvl88xKXxMOEOoUWkTpXRoaElhN7RYo/view?usp=sharing"
file_id = file_url.split("/")[-2]

# 파일 다운로드
url = f"https://drive.google.com/uc?id={file_id}&export=download"
response = requests.get(url)
content = response.content

# CSV 파일 읽고 정보 확인
bcancer_df = pd.read_csv(io.StringIO(content.decode('utf-8')))
bcancer_df

In [ ]:
bcancer_df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
target = LE.fit_transform(bcancer_df['diagnosis'])

bcancer_df.drop('diagnosis', axis=1, inplace=True)
bcancer_df['diagnosis'] = target

In [ ]:

features=bcancer_df.drop(['diagnosis','id','Unnamed: 32'],axis=1)


In [ ]:
import numpy as np

labels = bcancer_df['diagnosis'].to_numpy()

X_train,X_test,y_train,y_test=train_test_split(features,labels,test_size=0.2)

# Run individual base models

In [ ]:
dtree = tree.DecisionTreeClassifier(
    criterion="gini", max_depth=8, random_state=35)

dtree = dtree.fit(X_train, y_train)
dtree_predicted = dtree.predict(X_test)

knn = KNeighborsClassifier(n_neighbors=15).fit(X_train, y_train)
knn_predicted = knn.predict(X_test)

svm = SVC(kernel='rbf',C=.1, gamma=0.003,probability=True, random_state=35).fit(X_train, y_train)
svm_predicted = svm.predict(X_test)
svm_predicted

In [ ]:
print("[accuarcy]")
print("d-tree: ",accuracy_score(y_test, dtree_predicted))
print("knn   : ",accuracy_score(y_test, knn_predicted))
print("svm   : ",accuracy_score(y_test, svm_predicted))

# Get predict_proba data via SVM

In [ ]:
svm_probability=svm.predict_proba(X_test)
print(svm_probability.round(2))

# Hard voting

In [ ]:
voting_clf = VotingClassifier(estimators=[
    ('decision_tree', dtree), ('knn', knn), ('svm', svm)], 
    weights=[1,1,1], voting='hard').fit(X_train, y_train)
hard_voting_predicted = voting_clf.predict(X_test)
accuracy_score(y_test, hard_voting_predicted)

# Soft voting

In [ ]:
voting_clf = VotingClassifier(estimators=[
    ('decision_tree', dtree), ('knn', knn), ('svm', svm)], 
    weights=[1,1,1], voting='soft').fit(X_train, y_train)
soft_voting_predicted = voting_clf.predict(X_test)
accuracy_score(y_test, soft_voting_predicted)

# More close look into voting data

In [ ]:
y_test

In [ ]:
hard_voting_predicted 

In [ ]:
soft_voting_predicted

# Visualize accuracy scores for individual models and ensembling

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

x = np.arange(5)
plt.bar(x, height= [accuracy_score(y_test, dtree_predicted),
                    accuracy_score(y_test, knn_predicted),
                    accuracy_score(y_test, svm_predicted),
                    accuracy_score(y_test, hard_voting_predicted),
                    accuracy_score(y_test, soft_voting_predicted)])
plt.xticks(x, ['decision tree','knn','svm','hard voting','soft voting']);